In [1]:
import numpy as np
from pathlib import Path
import napari
from tqdm import tqdm
import pandas as pd
from cellpose import models, core, io
import pyclesperanto_prototype as cle 
from skimage.measure import regionprops_table
from skimage.segmentation import relabel_sequential
from skimage.morphology import remove_small_objects
from utils import list_images, read_image, extract_scaling_metadata

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose models
model = models.CellposeModel(gpu=True, model_type="cyto3") 

2026-01-22 14:34:23,549 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2026-01-22 14:34:23,549 [INFO] 
cellpose version: 	3.1.1.3 
platform:       	win32 
python version: 	3.10.19 
torch version:  	2.5.0
2026-01-22 14:34:23,597 [INFO] ** TORCH CUDA version installed and working. **
2026-01-22 14:34:23,597 [INFO] >> cyto3 << model set to be used
2026-01-22 14:34:23,597 [INFO] ** TORCH CUDA version installed and working. **
2026-01-22 14:34:23,597 [INFO] >>>> using GPU (CUDA)
2026-01-22 14:34:23,631 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cyto3
2026-01-22 14:34:23,690 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [2]:
# Define the markers you wish to analyze
markers = [("Occludin_RFP", 0, "membrane"), ("Claudin_FITC", 1, "membrane")]

In [3]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
#directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns")
directory_path = Path("./raw_data")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['raw_data\\B2.nd2']

In [4]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id from filename
well_id = filename.split("_")[0]

# Extract x,y,z scaling from .nd2 file metadata in order feed the Z pixel size / XY pixel size ratio into Cellpose
pixel_size_x, pixel_size_y, voxel_size_z = extract_scaling_metadata(image)

assert pixel_size_x == pixel_size_y
z_to_xy_ratio = voxel_size_z / pixel_size_x
print(z_to_xy_ratio)


Image analyzed: B2
Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 2.000 µm
3.017451031446116


In [5]:
#TODO: Extract number of multipos and loop through that range (store position for data extraction)
# Open one of the multipositions in the img file
img = img[3]
# Input shape for next step has to be C, Z, Y, X
img = img.transpose(1, 0, 2, 3)

# Keep only 3 meaningful input fluorescence channels to mimick Cellpose GUI normalization
# 0 membrane marker, 2 nuclei, 3 Cellmask
cellpose_input = img[[0, 2, 3],:,:,:]
img.shape

(5, 21, 1360, 1360)

In [6]:
cytoplasm_labels, flows, styles = model.eval(
    cellpose_input,                # numpy array
    channels=[3, 2],               # channels=[cyto_chan, nuclear_chan]
    diameter=20,                   # in pixels (XY), checked with GUI
    do_3D=True,
    anisotropy=z_to_xy_ratio,      # Z pixel size / XY pixel size
    normalize=True,
    flow_threshold=0.4,
    cellprob_threshold=0.0,
    min_size=15,
)
del cellpose_input

2026-01-22 14:34:25,240 [INFO] multi-stack tiff read in as having 21 planes 3 channels
2026-01-22 14:34:25,465 [INFO] resizing 3D image with rescale=1.50 and anisotropy=3.017451031446116
2026-01-22 14:34:26,677 [INFO] running YX: 95 planes of size (2040, 2040)
2026-01-22 14:35:02,406 [INFO] 100%|##########| 95/95 [00:35<00:00,  2.66it/s]
2026-01-22 14:35:04,581 [INFO] running ZY: 2040 planes of size (95, 2040)
2026-01-22 14:35:40,953 [INFO] 100%|##########| 2040/2040 [00:36<00:00, 56.09it/s]
2026-01-22 14:35:42,503 [INFO] running ZX: 2040 planes of size (95, 2040)
2026-01-22 14:36:28,540 [INFO] 100%|##########| 2040/2040 [00:46<00:00, 44.31it/s]
2026-01-22 14:36:33,328 [INFO] resizing 3D flows and cellprob to original image size
2026-01-22 14:36:34,537 [INFO] network run in 129.07s
2026-01-22 14:36:36,693 [INFO] masks created in 1.99s


In [7]:
# Initialize Napari Viewer
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)
viewer.add_labels(cytoplasm_labels)

<Labels layer 'cytoplasm_labels' at 0x1c6a3c16b00>

In [37]:
# Create a dictionary containing all image descriptors
#TODO: Add multiposition index during BP
descriptor_dict = {
            "filename": filename,
            "well_id": well_id,
            }

props_list = []

# Loop through markers and extract 
for marker_name, ch_nr, location in markers:
    print(f"Analyzing channel: {marker_name}")

    #TODO: Region cell or region membrane, generate membrane and extract info from that location
    # membranes = cle.reduce_labels_to_label_edges(cytoplasm_labels)
    props = regionprops_table(label_image=cytoplasm_labels,
                            intensity_image=img[ch_nr],
                            properties=[
                                "label",
                                "area",
                                "intensity_mean",
                                "intensity_min",
                                "intensity_max",
                                "intensity_std",
                            ],
                        )
    
    # Convert to dataframe
    props_df = pd.DataFrame(props)

    # Rename intensity_mean column to indicate the specific image
    prefix = f"{location}_{marker_name}"

    rename_map = {
        "intensity_mean": f"{prefix}_mean_int", # concentration proxy
        "intensity_min":  f"{prefix}_min_int",
        "intensity_max":  f"{prefix}_max_int",
        "intensity_std":  f"{prefix}_std_int",
    }

    props_df.rename(columns=rename_map, inplace=True)

    # Max / mean ratio (puncta vs diffuse signal)
    props_df[f"{prefix}_max_mean_ratio"] = (props_df[f"{prefix}_max_int"] /props_df[f"{prefix}_mean_int"])
    # Total marker content per cell
    props_df[f"{prefix}_sum_int"] = (props_df[f"{prefix}_mean_int"] * props_df["area"])

    # Append each props_df to props_list
    props_list.append(props_df)

    # Initialize the df with the first df in the list
    props_df = props_list[0]
    # Start looping from the second df in the list
    for df in props_list[1:]:
        props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0    
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

Analyzing channel: Occludin_RFP
Analyzing channel: Claudin_FITC


In [38]:
props_df

,filename,well_id,label,area_x,membrane_Occludin_RFP_mean_int,membrane_Occludin_RFP_min_int,membrane_Occludin_RFP_max_int,membrane_Occludin_RFP_std_int,membrane_Occludin_RFP_max_mean_ratio,membrane_Occludin_RFP_sum_int,area_y,membrane_Claudin_FITC_mean_int,membrane_Claudin_FITC_min_int,membrane_Claudin_FITC_max_int,membrane_Claudin_FITC_std_int,membrane_Claudin_FITC_max_mean_ratio,membrane_Claudin_FITC_sum_int
0,B2,B2,1,3424.0,277.589661,136.0,983.0,80.776562,3.541198,950467.0,3424.0,532.482185,233.0,1256.0,164.412146,2.358764,1823219.0
1,B2,B2,2,3256.0,204.658784,105.0,1087.0,73.148733,5.311279,666369.0,3256.0,227.188575,110.0,708.0,72.642705,3.116354,739726.0
2,B2,B2,3,4189.0,249.182144,132.0,904.0,73.875658,3.627868,1043824.0,4189.0,495.893053,182.0,1701.0,178.672270,3.430175,2077296.0
3,B2,B2,4,3213.0,302.078120,136.0,930.0,87.169359,3.078674,970577.0,3213.0,534.435730,167.0,1246.0,175.541747,2.331431,1717142.0
4,B2,B2,5,1523.0,172.711753,108.0,363.0,43.761864,2.101768,263040.0,1523.0,336.382797,91.0,1959.0,269.876845,5.823722,512311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,B2,B2,1294,91.0,152.714286,121.0,195.0,16.148545,1.276894,13897.0,91.0,374.241758,249.0,543.0,67.072266,1.450934,34056.0
1294,B2,B2,1295,33.0,160.333333,129.0,201.0,17.862550,1.253638,5291.0,33.0,463.272727,286.0,782.0,122.479008,1.687991,15288.0
1295,B2,B2,1296,34.0,211.911765,146.0,281.0,34.715963,1.326024,7205.0,34.0,579.764706,419.0,700.0,65.448704,1.207386,19712.0
1296,B2,B2,1297,41.0,194.463415,147.0,254.0,28.515377,1.306158,7973.0,41.0,597.024390,432.0,806.0,78.999687,1.350029,24478.0


In [10]:
# Segment whole organoids from input image using individual Cellpose labels as starting point

# Maximum projection across z-axis to flatten 3D labels into 2D space
mip_labels = np.max(cytoplasm_labels, axis=0)
viewer.add_labels(mip_labels)

# Merge touching labels to establish first organoid entities
merged_mip = cle.merge_touching_labels(mip_labels)
viewer.add_labels(merged_mip)

# Dilation-Erosion cycle to close holes
dilated_labels = cle.dilate_labels(merged_mip, radius=5)
eroded_labels = cle.erode_labels(dilated_labels, radius=1)
viewer.add_labels(eroded_labels)

# Pull from GPU in order to filter out small org and relabel using skimage
org_labels = cle.pull(eroded_labels)
org_labels = remove_small_objects(org_labels, min_size=5000)

# Relabel starting from 1
organoid_labels = relabel_sequential(org_labels)[0]
viewer.add_labels(organoid_labels)

<Labels layer 'organoid_labels' at 0x1c72cc5a050>

In [11]:
membranes = cle.reduce_labels_to_label_edges(cytoplasm_labels)
viewer.add_labels(membranes)

<Labels layer 'membranes' at 0x1c765550880>

In [39]:
# Map each cell label to each corresponding organoid
def map_small_to_big(labels_small, labels_big):
    mask = labels_small > 0
    pairs = np.stack([
        labels_small[mask],
        labels_big[mask]
    ], axis=1)

    # remove background overlaps
    pairs = pairs[pairs[:, 1] > 0]

    mapping = {}
    for s, b in pairs:
        mapping.setdefault(int(b), set()).add(int(s))

    return mapping

mapping = map_small_to_big(mip_labels, organoid_labels)

# Invert mapping to map to props_df
small_to_big = {}
for b, smalls in mapping.items():
    for s in smalls:
        small_to_big.setdefault(s, set()).add(b)

In [40]:
# Add organoid column to props_df
props_df["organoid"] = (
    props_df["label"]
    .map(lambda s: next(iter(small_to_big[s]))
         if s in small_to_big and len(small_to_big[s]) == 1
         else 0)
    .astype(int)
)

# Reorder so it appears after well_id and before label
cols = list(props_df.columns)
cols.insert(cols.index("well_id") + 1, cols.pop(cols.index("organoid")))
props_df = props_df[cols]


props_df

,filename,well_id,organoid,label,area_x,membrane_Occludin_RFP_mean_int,membrane_Occludin_RFP_min_int,membrane_Occludin_RFP_max_int,membrane_Occludin_RFP_std_int,membrane_Occludin_RFP_max_mean_ratio,membrane_Occludin_RFP_sum_int,area_y,membrane_Claudin_FITC_mean_int,membrane_Claudin_FITC_min_int,membrane_Claudin_FITC_max_int,membrane_Claudin_FITC_std_int,membrane_Claudin_FITC_max_mean_ratio,membrane_Claudin_FITC_sum_int
0,B2,B2,4,1,3424.0,277.589661,136.0,983.0,80.776562,3.541198,950467.0,3424.0,532.482185,233.0,1256.0,164.412146,2.358764,1823219.0
1,B2,B2,0,2,3256.0,204.658784,105.0,1087.0,73.148733,5.311279,666369.0,3256.0,227.188575,110.0,708.0,72.642705,3.116354,739726.0
2,B2,B2,1,3,4189.0,249.182144,132.0,904.0,73.875658,3.627868,1043824.0,4189.0,495.893053,182.0,1701.0,178.672270,3.430175,2077296.0
3,B2,B2,1,4,3213.0,302.078120,136.0,930.0,87.169359,3.078674,970577.0,3213.0,534.435730,167.0,1246.0,175.541747,2.331431,1717142.0
4,B2,B2,0,5,1523.0,172.711753,108.0,363.0,43.761864,2.101768,263040.0,1523.0,336.382797,91.0,1959.0,269.876845,5.823722,512311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,B2,B2,17,1294,91.0,152.714286,121.0,195.0,16.148545,1.276894,13897.0,91.0,374.241758,249.0,543.0,67.072266,1.450934,34056.0
1294,B2,B2,0,1295,33.0,160.333333,129.0,201.0,17.862550,1.253638,5291.0,33.0,463.272727,286.0,782.0,122.479008,1.687991,15288.0
1295,B2,B2,18,1296,34.0,211.911765,146.0,281.0,34.715963,1.326024,7205.0,34.0,579.764706,419.0,700.0,65.448704,1.207386,19712.0
1296,B2,B2,18,1297,41.0,194.463415,147.0,254.0,28.515377,1.306158,7973.0,41.0,597.024390,432.0,806.0,78.999687,1.350029,24478.0


In [41]:
# Sanity checks

# Cells with no organoid
n_orphans = (props_df["organoid"] == 0).sum()

# Cells mapped to non-existing organoids
bad = ~props_df["organoid"].isin(np.unique(organoid_labels))
if bad.any():
    bad_rows = props_df.loc[bad]
    print("Cells mapped to non-existing organoids:")
    display(bad_rows)

# Cells assigned to multiple organoids
n_multi_parent = (
    props_df
    .groupby("label")["organoid"]
    .nunique()
    .gt(1)
    .sum()
)


In [42]:
#Extract area information at an organoid level and merge with the existing props_df

organoid_props = regionprops_table(label_image=organoid_labels,
                            properties=[
                                "label",
                                "area",           # organoid size (2D MIP)
                                "perimeter",      # boundary complexity
                                "eccentricity",   # round (0) → elongated (1)
                                "solidity",       # filled vs lobed (area / convex area)
                                "extent",         # area / bounding box area
                            ],
                        )
    
# Convert to dataframe
organoids_props_df = pd.DataFrame(organoid_props)

# Rename intensity_mean column to indicate the specific image
prefix = "organoid"

rename_map = {
    "label": "organoid",
    "area": f"{prefix}_area",
    "perimeter": f"{prefix}_perimeter",
    "eccentricity": f"{prefix}_eccentricity",
    "solidity": f"{prefix}_solidity",
    "extent": f"{prefix}_extent"
}

organoids_props_df.rename(columns=rename_map, inplace=True)

organoids_props_df

,organoid,organoid_area,organoid_perimeter,organoid_eccentricity,organoid_solidity,organoid_extent
0,1,6346.0,306.208153,0.448912,0.972120,0.810887
1,2,9869.0,406.391919,0.389172,0.941788,0.734301
2,3,9406.0,427.605122,0.414318,0.900182,0.687573
3,4,16824.0,519.906638,0.748167,0.984723,0.824060
4,5,11431.0,406.333044,0.547634,0.982045,0.783374
5,6,5197.0,383.663997,0.799762,0.821920,0.571727
6,7,8834.0,375.747258,0.609818,0.967473,0.736167
7,8,5152.0,434.676190,0.587404,0.804623,0.690432
8,9,5617.0,342.835570,0.666878,0.891446,0.700461
9,10,11976.0,426.090404,0.454523,0.970109,0.779332


In [ ]:
# Merge organoid_props and cytoplasm_props Dataframes

final_df = props_df.merge(
    organoids_props_df,
    how="left",
    on="organoid"
)

final_df

,filename,well_id,organoid,label,area_x,membrane_Occludin_RFP_mean_int,membrane_Occludin_RFP_min_int,membrane_Occludin_RFP_max_int,membrane_Occludin_RFP_std_int,membrane_Occludin_RFP_max_mean_ratio,...,membrane_Claudin_FITC_min_int,membrane_Claudin_FITC_max_int,membrane_Claudin_FITC_std_int,membrane_Claudin_FITC_max_mean_ratio,membrane_Claudin_FITC_sum_int,organoid_area,organoid_perimeter,organoid_eccentricity,organoid_solidity,organoid_extent
0,B2,B2,4,1,3424.0,277.589661,136.0,983.0,80.776562,3.541198,...,233.0,1256.0,164.412146,2.358764,1823219.0,16824.0,519.906638,0.748167,0.984723,0.824060
1,B2,B2,0,2,3256.0,204.658784,105.0,1087.0,73.148733,5.311279,...,110.0,708.0,72.642705,3.116354,739726.0,NaN,NaN,NaN,NaN,NaN
2,B2,B2,1,3,4189.0,249.182144,132.0,904.0,73.875658,3.627868,...,182.0,1701.0,178.672270,3.430175,2077296.0,6346.0,306.208153,0.448912,0.972120,0.810887
3,B2,B2,1,4,3213.0,302.078120,136.0,930.0,87.169359,3.078674,...,167.0,1246.0,175.541747,2.331431,1717142.0,6346.0,306.208153,0.448912,0.972120,0.810887
4,B2,B2,0,5,1523.0,172.711753,108.0,363.0,43.761864,2.101768,...,91.0,1959.0,269.876845,5.823722,512311.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,B2,B2,17,1294,91.0,152.714286,121.0,195.0,16.148545,1.276894,...,249.0,543.0,67.072266,1.450934,34056.0,10603.0,523.587878,0.496081,0.910754,0.713142
1294,B2,B2,0,1295,33.0,160.333333,129.0,201.0,17.862550,1.253638,...,286.0,782.0,122.479008,1.687991,15288.0,NaN,NaN,NaN,NaN,NaN
1295,B2,B2,18,1296,34.0,211.911765,146.0,281.0,34.715963,1.326024,...,419.0,700.0,65.448704,1.207386,19712.0,27678.0,681.043723,0.339649,0.962144,0.779223
1296,B2,B2,18,1297,41.0,194.463415,147.0,254.0,28.515377,1.306158,...,432.0,806.0,78.999687,1.350029,24478.0,27678.0,681.043723,0.339649,0.962144,0.779223
